In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error
import numpy as np

In [ ]:
# Sample customer behavior data
# Columns: 'customer_name', 'product_id', 'purchase'
data = {
    'customer_name': ['Alice', 'Bob', 'Charlie', 'David', 'Alice', 'Bob', 'David', 'Charlie'],
    'product_id': [101, 102, 103, 104, 102, 101, 103, 101],
    'purchase': [1, 1, 1, 1, 1, 0, 1, 0]
}

# Convert the data into a DataFrame
df = pd.DataFrame(data)

# Create a pivot table for the recommendation engine
pivot_table = df.pivot_table(index='customer_name', columns='product_id', values='purchase', fill_value=0)

# Create a similarity matrix using cosine similarity
similarity_matrix = cosine_similarity(pivot_table)

# Create a DataFrame for the similarity matrix
similarity_df = pd.DataFrame(similarity_matrix, index=pivot_table.index, columns=pivot_table.index)

def recommend_products(customer_name, pivot_table, similarity_df, n_recommendations=3):
    # Find the similar customers
    similar_customers = similarity_df[customer_name].sort_values(ascending=False)[1:]

    # Weighted recommendations
    recommendations = {}

    for similar_customer, similarity in similar_customers.items():
        similar_customer_purchases = pivot_table.loc[similar_customer]
        for product, purchase in similar_customer_purchases.items():
            if pivot_table.loc[customer_name, product] == 0:  # Only recommend products the customer hasn't purchased
                if product not in recommendations:
                    recommendations[product] = similarity * purchase
                else:
                    recommendations[product] += similarity * purchase

    # Sort recommendations by score and return top n recommendations
    recommended_products = sorted(recommendations.items(), key=lambda x: x[1], reverse=True)[:n_recommendations]

    return [product for product, score in recommended_products]

# Example usage
customer_name = 'Alice'
recommended_products = recommend_products(customer_name, pivot_table, similarity_df)
print(f"Recommended products for {customer_name}: {recommended_products}")

customer_name = 'Bob'
recommended_products = recommend_products(customer_name, pivot_table, similarity_df)
print(f"Recommended products for {customer_name}: {recommended_products}")

Recommended products for Alice: [103, 104]
Recommended products for Bob: [101, 103, 104]
